In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

print(tf.test.is_gpu_available())

train_df = pd.read_csv('task3_2021.training.csv')
test_df = pd.read_csv('task3_2021.test.csv')
n_pre_train_y = train_df['target']
n_pre_train_x = train_df.drop(['target'], axis=1)
n_pre_test_x = test_df
# display(n_pre_train_x.head(), n_pre_train_y.head(), n_pre_train_y.nunique(), n_pre_test_x.head())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2021-12-21 02:30:16.498629: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-21 02:30:19.358969: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-21 02:30:19.359467: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-21 02:30:19.452364: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

True


,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18
0,14.557571,11.846020,15.020207,2.623484,15.538517,2.341611,5.829028,7.586709,13.331953,8.304782,-8.980424,-1.978135,-9.056215,0.601958,-3.990212,F,F,A,-8.435632
1,17.031884,4.314536,19.694838,15.264186,7.054585,0.382333,4.650885,11.331481,12.492329,12.877221,-1.788222,-12.520625,1.183564,2.312428,-5.997316,F,A,F,13.314504
2,24.986272,6.139161,21.949963,5.590123,16.445212,2.213986,2.393948,16.885378,8.114116,20.133208,12.148553,-11.400213,15.273799,2.714855,6.574276,A,C,F,8.964849
3,7.584971,11.598161,9.793109,11.873020,6.645073,3.346882,4.445008,13.218360,8.495786,17.403693,-3.371434,-1.329931,4.307364,1.217894,-1.730330,D,D,B,13.563612
4,11.625079,12.517998,15.522108,18.269750,1.508771,3.078033,5.224333,18.335198,9.604704,19.732707,3.192207,2.195230,10.903963,2.317327,1.379398,B,C,B,-8.590211


0    B
1    G
2    G
3    H
4    F
Name: target, dtype: object

10

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18
0,12.981599,14.926975,14.055879,12.940302,6.547128,3.765283,4.911310,12.816209,12.321573,17.026134,-4.435416,-3.338954,5.145203,0.790691,-2.833418,E,D,A,-3.198949
1,26.898603,8.866915,20.358121,6.067027,13.505201,1.742242,3.731071,14.026740,13.686651,17.920196,3.398008,-4.949895,2.115480,1.839801,2.199004,F,F,E,10.497858
2,20.817003,5.789936,20.937223,8.249511,13.569874,0.060766,5.505705,7.102595,12.735801,7.195690,10.580991,-5.864917,-2.386709,2.558099,5.790495,F,F,F,-11.909326
3,9.994456,12.242201,7.906568,10.475264,14.327991,0.856716,4.754537,26.673467,6.219800,21.140358,-11.304655,-8.923086,5.620824,1.436804,-6.877449,C,B,A,-0.256432
4,8.640119,11.944469,14.674074,13.246914,5.013325,2.332369,4.226819,6.004056,11.400005,6.236202,1.708020,12.079740,16.079740,1.037320,5.126897,A,C,D,12.142437


time: 45 s (started: 2021-12-21 02:29:47 +00:00)


In [2]:
def oh_pre(x):
    for k in ['f15', 'f16', 'f17']:
        tmp_one_hot_k =  pd.get_dummies(x[k], prefix=f"{k}")
        x = pd.concat([x, tmp_one_hot_k],axis=1)
        x = x.drop([k], axis=1)
    return x

oh_pre_train_x = oh_pre(n_pre_train_x)
oh_pre_test_x = oh_pre(n_pre_test_x)
scaler = StandardScaler()
train_x = scaler.fit_transform(oh_pre_train_x)
test_x = scaler.transform(oh_pre_test_x)

ohenc = OneHotEncoder(handle_unknown="ignore", sparse=False)
train_y = ohenc.fit_transform(n_pre_train_y.values.reshape(-1,1))
# mse = make_scorer(mean_squared_error, squared=False)nunique

time: 402 ms (started: 2021-12-21 02:30:33 +00:00)


In [5]:
class DNN(tf.keras.Model):
  def __init__(self):
    super(DNN,self).__init__()
    self.dense1 = tf.keras.layers.Dense(48, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(48, activation=tf.nn.relu)
    self.dense3 = tf.keras.layers.Dense(48, activation=tf.nn.relu)
    self.output_layer = tf.keras.layers.Dense(10, activation=tf.nn.softmax)

  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)
    x = self.dense3(x)
    return self.output_layer(x)

def scheduler(epoch, lr):
    if epoch < 100:
        return lr
    elif epoch < 90:
        return lr*tf.math.exp(1e-1)


callback_early = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=10,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

callback_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)

batch_size = 100
epochs = 1000
opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
model = DNN()
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[callback_lr,callback_early], verbose=1)
model.summary()

Epoch 1/1000
90/90 [==============================] - 8s 33ms/step - loss: 2.0402 - accuracy: 0.2239 - val_loss: 1.6177 - val_accuracy: 0.3370 - lr: 0.0010
Epoch 2/1000
90/90 [==============================] - 1s 16ms/step - loss: 1.2730 - accuracy: 0.4888 - val_loss: 1.0135 - val_accuracy: 0.5550 - lr: 0.0010
Epoch 3/1000
90/90 [==============================] - 1s 15ms/step - loss: 0.8089 - accuracy: 0.6812 - val_loss: 0.6826 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 4/1000
90/90 [==============================] - 1s 16ms/step - loss: 0.5846 - accuracy: 0.7797 - val_loss: 0.5198 - val_accuracy: 0.8160 - lr: 0.0010
Epoch 5/1000
90/90 [==============================] - 1s 15ms/step - loss: 0.4669 - accuracy: 0.8259 - val_loss: 0.4298 - val_accuracy: 0.8400 - lr: 0.0010
Epoch 6/1000
90/90 [==============================] - 1s 15ms/step - loss: 0.3944 - accuracy: 0.8498 - val_loss: 0.3727 - val_accuracy: 0.8590 - lr: 0.0010
Epoch 7/1000
90/90 [==============================] - 1s 15ms/st

In [6]:
pred = model.predict(test_x)
pred

array([2.2978238e-15, 1.0525484e-11, 8.8355155e-04, ..., 1.7472745e-04,
       9.9976677e-01, 5.8548019e-05], dtype=float32)

time: 35.4 s (started: 2021-12-21 02:43:57 +00:00)


In [16]:
from pandas import RangeIndex

nn_pred = model.predict(test_x)


pred = np.where(nn_pred<0.5, 0, 1)
output = ohenc.inverse_transform(pred)
check = (output==None).sum()
if check==0:
    pd.Series(output, index=RangeIndex(1, 2001), name='Predicted').to_csv('ans_nn_1.csv',index_label='id')

0

time: 1.29 s (started: 2021-12-21 02:49:23 +00:00)
